In [ ]:
!pip install transformers bitsandbytes accelerate

In [2]:
token=""
model_id="epfl-llm/meditron-7b"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
model = AutoModelForCausalLM.from_pretrained(model_id, token=token, load_in_8bit=True)

In [4]:
from transformers import TextStreamer

prompt = """# Question 1: Please identify the genetic biomarkers in the following chunk of text (marked inside "----")
----
The subject has a pathologic diagnosis of non-small cell lung carcinoma that is metastatic or unresectable  
Documented presence
* GROUP A: kinesin family member 5B (KIF5B)-RET or related variant RET fusions
* GROUP B: any of the following aberrations
** NTRK fusion
** MET overexpression, amplification, or mutation
** AXL overexpression, amplification, or mutation
* GROUP C: ROS1 fusion
* GROUP D: RET-fusion post-progression on selective RET inhibitor  
Measurable disease by RECIST1.1  
The subject has a Karnofsky performance status of > 70%  
Absolute neutrophil count (ANC) >= 1500/mm^3 without colony stimulating factor support  
Platelets >= 100,000/mm^3  
Hemoglobin >= 9 g/dL  
Bilirubin =< 1.5 x the upper limit of normal (ULN); for subjects with known Gilbert's disease, bilirubin =< 3.0 mg/dL  
Serum creatinine =< 1.5 x ULN, or creatinine clearance (CrCl) >= 30 mL/min; for creatinine clearance estimation, the Cockcroft and Gault equation should be used  
Alanine aminotransferase (ALT) and aspartate aminotransferase (AST) =< 3.0 x ULN if no liver involvement, or =< 5 x ULN with liver involvement  
Urine protein/creatinine ratio (UPCR) =< 1 mg/mg (113.2 mg/mmol) creatinine or 24-hour (hr) urine protein of < 1 g  
Serum phosphorus, magnesium, and potassium >= lower limit of normal (LLN) after adequate supplementation if necessary  
The subject is capable of understanding and complying with the protocol requirements and has signed the informed consent document  
Sexually active subjects (men and women) must agree to use medically accepted barrier methods of contraception (e.g., male or female condom) during the course of the study and for 4 months after the last dose of study drug(s), even if oral contraceptives are also used; all subjects of reproductive potential must agree to use both a barrier method and a second method of birth control; women of childbearing potential must have a negative pregnancy test at screening
----
# Answer: 
"""
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Memory management

In [4]:
# /home/ec2-user-user/sagemaker shows the EBS volume size
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         16G     0   16G   0% /dev
tmpfs            16G  132K   16G   1% /dev/shm
tmpfs            16G  680K   16G   1% /run
tmpfs            16G     0   16G   0% /sys/fs/cgroup
/dev/nvme0n1p1  135G  100G   36G  74% /
tmpfs           3.1G     0  3.1G   0% /run/user/0
/dev/nvme2n1    4.8G  128K  4.6G   1% /home/ec2-user/SageMaker
tmpfs           3.1G     0  3.1G   0% /run/user/1000
tmpfs           3.1G     0  3.1G   0% /run/user/1002
tmpfs           3.1G     0  3.1G   0% /run/user/1001


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            30G        1.9G         12G         12M         16G         28G
Swap:            0B          0B          0B


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
import torch
import gc
from numba import cuda
# cuda.select_device(0)
# cuda.close()

# model = None
# tokenizer = None
# inputs = None
# outputs = None
# print(cuda)

torch.cuda.empty_cache()
# gc.collect()
!nvidia-smi

Fri Dec 29 22:03:32 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   29C    P0              38W /  70W |   8537MiB / 15360MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Greedy search

In [5]:
hyperparams = {
    'streamer': TextStreamer(tokenizer),
    'max_new_tokens': 200,
    'pad_token_id': tokenizer.eos_token_id
}
outputs = model.generate(**inputs, **hyperparams)

<s> # Question 1: Please identify the genetic biomarkers in the following chunk of text (marked inside "----")
----
The subject has a pathologic diagnosis of non-small cell lung carcinoma that is metastatic or unresectable  
Documented presence
* GROUP A: kinesin family member 5B (KIF5B)-RET or related variant RET fusions
* GROUP B: any of the following aberrations
** NTRK fusion
** MET overexpression, amplification, or mutation
** AXL overexpression, amplification, or mutation
* GROUP C: ROS1 fusion
* GROUP D: RET-fusion post-progression on selective RET inhibitor  
Measurable disease by RECIST1.1  
The subject has a Karnofsky performance status of > 70%  
Absolute neutrophil count (ANC) >= 1500/mm^3 without colony stimulating factor support  
Platelets >= 100,000/mm^3  
Hemoglobin >= 9 g/dL  
Bilirubin =< 1.5 x the upper limit of normal (ULN); for subjects with known Gilbert's disease, bilirubin =< 3.0 mg/dL  
Serum creatinine =< 1.5 x ULN, or creatinine clearance (CrCl) >= 30 mL/min

# John's hyperparams (llama2)

In [9]:
samplingparams = {
    'do_sample': True,
    'temperature': 0.6,
    'top_p': 0.5,
    'top_k': 35,
}
hyperparams = {
    'streamer': TextStreamer(tokenizer),
    'max_new_tokens': 200,
    'pad_token_id': tokenizer.eos_token_id
}
outputs = model.generate(**inputs, **samplingparams, **hyperparams)

<s> # Question 1: Please identify the genetic biomarkers in the following chunk of text (marked inside "----")
----
The subject has a pathologic diagnosis of non-small cell lung carcinoma that is metastatic or unresectable  
Documented presence
* GROUP A: kinesin family member 5B (KIF5B)-RET or related variant RET fusions
* GROUP B: any of the following aberrations
** NTRK fusion
** MET overexpression, amplification, or mutation
** AXL overexpression, amplification, or mutation
* GROUP C: ROS1 fusion
* GROUP D: RET-fusion post-progression on selective RET inhibitor  
Measurable disease by RECIST1.1  
The subject has a Karnofsky performance status of > 70%  
Absolute neutrophil count (ANC) >= 1500/mm^3 without colony stimulating factor support  
Platelets >= 100,000/mm^3  
Hemoglobin >= 9 g/dL  
Bilirubin =< 1.5 x the upper limit of normal (ULN); for subjects with known Gilbert's disease, bilirubin =< 3.0 mg/dL  
Serum creatinine =< 1.5 x ULN, or creatinine clearance (CrCl) >= 30 mL/min

# Top-p (nucleus) sampling

In [10]:
samplingparams = {
    'do_sample': True,
    'top_p': 0.95,
    'top_k': 0
}
hyperparams = {
    'streamer': TextStreamer(tokenizer),
    'max_new_tokens': 200,
    'pad_token_id': tokenizer.eos_token_id
}
outputs = model.generate(**inputs, **samplingparams, **hyperparams)

<s> # Question 1: Please identify the genetic biomarkers in the following chunk of text (marked inside "----")
----
The subject has a pathologic diagnosis of non-small cell lung carcinoma that is metastatic or unresectable  
Documented presence
* GROUP A: kinesin family member 5B (KIF5B)-RET or related variant RET fusions
* GROUP B: any of the following aberrations
** NTRK fusion
** MET overexpression, amplification, or mutation
** AXL overexpression, amplification, or mutation
* GROUP C: ROS1 fusion
* GROUP D: RET-fusion post-progression on selective RET inhibitor  
Measurable disease by RECIST1.1  
The subject has a Karnofsky performance status of > 70%  
Absolute neutrophil count (ANC) >= 1500/mm^3 without colony stimulating factor support  
Platelets >= 100,000/mm^3  
Hemoglobin >= 9 g/dL  
Bilirubin =< 1.5 x the upper limit of normal (ULN); for subjects with known Gilbert's disease, bilirubin =< 3.0 mg/dL  
Serum creatinine =< 1.5 x ULN, or creatinine clearance (CrCl) >= 30 mL/min

# Contrastive search

In [7]:
samplingparams = {
    'penalty_alpha': 0.5,
    'top_k': 4
}
hyperparams = {
    'streamer': TextStreamer(tokenizer),
    'max_new_tokens': 200,
    'pad_token_id': tokenizer.eos_token_id
}
outputs = model.generate(**inputs, **samplingparams, **hyperparams)

<s> # Question 1: Please identify the genetic biomarkers in the following chunk of text (marked inside "----")
----
The subject has a pathologic diagnosis of non-small cell lung carcinoma that is metastatic or unresectable  
Documented presence
* GROUP A: kinesin family member 5B (KIF5B)-RET or related variant RET fusions
* GROUP B: any of the following aberrations
** NTRK fusion
** MET overexpression, amplification, or mutation
** AXL overexpression, amplification, or mutation
* GROUP C: ROS1 fusion
* GROUP D: RET-fusion post-progression on selective RET inhibitor  
Measurable disease by RECIST1.1  
The subject has a Karnofsky performance status of > 70%  
Absolute neutrophil count (ANC) >= 1500/mm^3 without colony stimulating factor support  
Platelets >= 100,000/mm^3  
Hemoglobin >= 9 g/dL  
Bilirubin =< 1.5 x the upper limit of normal (ULN); for subjects with known Gilbert's disease, bilirubin =< 3.0 mg/dL  
Serum creatinine =< 1.5 x ULN, or creatinine clearance (CrCl) >= 30 mL/min

# Multinomial sampling / Beam-search

In [4]:
samplingparams = {
    # 'do_sample': True,
    'num_beams': 3,
    'early_stopping': True
}
hyperparams = {
    'streamer': TextStreamer(tokenizer) if samplingparams['num_beams'] <= 1 else None,
    'max_new_tokens': 200,
    'pad_token_id': tokenizer.eos_token_id
}
outputs = model.generate(**inputs, **samplingparams, **hyperparams)
if not hyperparams['streamer']:
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

# Question 1: Please identify the genetic biomarkers in the following chunk of text (marked inside "----")
----
The subject has a pathologic diagnosis of non-small cell lung carcinoma that is metastatic or unresectable  
Documented presence
* GROUP A: kinesin family member 5B (KIF5B)-RET or related variant RET fusions
* GROUP B: any of the following aberrations
** NTRK fusion
** MET overexpression, amplification, or mutation
** AXL overexpression, amplification, or mutation
* GROUP C: ROS1 fusion
* GROUP D: RET-fusion post-progression on selective RET inhibitor  
Measurable disease by RECIST1.1  
The subject has a Karnofsky performance status of > 70%  
Absolute neutrophil count (ANC) >= 1500/mm^3 without colony stimulating factor support  
Platelets >= 100,000/mm^3  
Hemoglobin >= 9 g/dL  
Bilirubin =< 1.5 x the upper limit of normal (ULN); for subjects with known Gilbert's disease, bilirubin =< 3.0 mg/dL  
Serum creatinine =< 1.5 x ULN, or creatinine clearance (CrCl) >= 30 mL/min; fo